# メタデータからデータ管理品質を検証する

研究データに付与されているメタデータを検証し、研究データ管理の品質をレポートします。  
検証結果のレポートは、この環境で確認したあとにGIN-forkへ同期するか破棄するか選択いただけます。  
GIN-forkへ同期する場合は、研究リポジトリの直下にありますvalidation_resultsフォルダでバージョン管理されます。

## 0. 研究リポジトリ名を確認する

以下のセルを実行すると、この実験実行環境で操作する研究リポジトリ名を確認できます。 

In [ ]:
import os
os.chdir('/home/jovyan/WORKFLOWS/FLOW/')
from util.scripts import utils
utils.show_name('green')

## 1. 研究プロジェクトのメタデータを取得する

In [ ]:
# http://dg01.dg.rcos.nii.ac.jp/api/v1/repos/:repo_id/:branch/metadata?token=6e7816fb21fe01c303d870bf6a4192ecf00548e2
# ユーザ認証時のtokenを保存しておいて良い？セキュリティ的には毎回認証してトークンを更新すべき？



In [ ]:
# 現在のブランチ名を取得する

result = !git branch --contains
current_branch = result[0].replace('* ', '')
current_branch

In [ ]:
# repository_id取得
import os
os.chdir(os.environ['HOME'])

from WORKFLOWS.utils.repository_id import repository_id
repo_id = repository_id.get_repo_id()
repo_id

## 2. メタデータをパッケージングする

RoGeneratorクラスのgenerate関数にrawメタデータを入力してro-crate-metadata.jsonを取得する。
余力あれば、エンティティを選択して検証するユースケースに対応する。  
**検証したいエンティティのIDリストをPKGに生成してもらう**  
v1.5ではGinMonitoringのID  
PKGに渡すパラメータが必要

## 3. 検証サービスに検証を依頼する

In [ ]:
import requests

try:
    req_response = requests.post("https://example.com/**/validate?entityIds=a&entityIds=b", json=crate_json)
    result = req_response.json
except:
    print('検証サービスに接続できません。')
else:
    if req_response.status_code == 200:
        request_id = result['request_id']
        print('正常に検証が依頼されました。次の処理にお進みください。\nリクエストID：' + request_id)
    elif req_response.status_code == 400:
        print('BadRequest')
        print(result['message'])
    elif req_response.status_code == 500:
        print('InternalServerError')
        print(result['message'])

## 4. 検証サービスから検証結果を取得する

In [ ]:
import requests
import time

while True:

    try:
        result_response = requests.get("https://example.com/*/" + request_id)
        result = result_response.json
    except:
        print('検証サービスに接続できません。')
    else:
        if result_response.status_code == 200:
            status = result['status']
            if status == 'UNKNOWN':
                print('リクエストID：' + request_id + 'は正常にリクエストできていません。')
                break
            elif any([status == 'QUEUED', status == 'RUNNING']):
                print('リクエストID：' + request_id + 'はまだ検証完了していません。時間をおいて再度確認します。)
                # アスケイドさんの単体テストで5sec待っていたのを参考に5秒に設定している
                time.sleep(5)
                continue
            elif status == 'COMPLETE':
                ro_crate = result['request']['roCrate']
                entity_ids = result['request']['entityIds']
                results = result['results']
                print('正常に検証結果を取得しました。次の処理にお進みください。')
                break
            elif any([status == 'FAILED', status == 'EXECUTOR_ERROR']):
                print('正常に検証できませんでした。')
                break
            elif status == 'CANCELING':
                print('リクエストID：' + request_id + 'は現在キャンセル中です。')
                break
            elif status == 'CANCELED':
                print('リクエストID：' + request_id + 'はキャンセルされました。')
                break

        elif result_response.status_code == 400:
            print('BadRequest')
            print(result['message'])
            break
        elif result_response.status_code == 500:
            print('InternalServerError')
            print(result['message'])
            break

## 5. 検証結果からユーザ向けUIを生成する　初めは出力するだけ


TBD

## 6. ユーザ向けUIへのリンクを出力する


In [ ]:
import json
import os
from IPython.display import display, HTML

!mkdir -p .tmp
os.chdir(os.environ['HOME'])
validation_result_file = '.tmp/' + request_id + '.json'
with open(validation_result_file, 'w') as file:
    json.dump(results, file, indent=4)
display(HTML('<a href=../../../../edit/' + validation_result_file + '>検証結果を確認する</a>'))

## 7. 検証結果をリポジトリに同期するかユーザに選択してもらう

## 8. 同期する場合は、結果を.tmpからvalidation_resultsに移動する

## 9. 同期処理を行う